In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
import time

In [2]:
# first date of new concept with 2 out of 12 (before 2 out of 10)
start_date = datetime.date(2022,3,25)

#### Friday and Tuesday are the draw days

In [3]:
fridays = []
date_m = start_date
while date_m <= datetime.date.today():
    nextFriday = date_m + datetime.timedelta(days=-date_m.weekday()+4, weeks=1)
    fridays.append(nextFriday)
    date_m = nextFriday

In [4]:
tuesdays = []
date_m = start_date
while date_m <= datetime.date.today():
    nextTuesdays = date_m + datetime.timedelta(days=-date_m.weekday()+1, weeks=1)
    tuesdays.append(nextTuesdays)
    date_m = nextTuesdays


#### Format the dates for the API call

In [5]:
date_list = [start_date] + tuesdays + fridays

In [6]:
date_list = [x.strftime("%Y-%m-%d") for x in date_list]

#### Function to call the API and return the json

In [7]:
def get_quotes(date):

    url = "https://www.eurojackpot.de/wlinfo/WL_InfoService"

    querystring = {"client":"jsn","gruppe":"ZahlenUndQuoten","ewGewsum":"ja","historie":"ja","spielart":"EJ","adg":"ja","lang":"de","datum":date}

    payload = ""
    headers = {
        "Accept": "application/json, text/plain, */*",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.134 Safari/537.36"
    }

    response = requests.request("GET", url, data=payload, headers=headers, params=querystring)

    classes = response.json()['auswertung']['quoten']['hauptlotterie']['ziehungen'][0]['gewinnklassen']
    five = response.json()['zahlen']['hauptlotterie']['ziehungen'][0]['zahlen']
    two = response.json()['zahlen']['hauptlotterie']['ziehungen'][1]['zahlen']

    return classes, five, two


#### Write the json data to a pandas dataframe and save it as .csv

In [8]:
df = pd.DataFrame(columns=['Datum', 'Klasse', 'Gewinn'])

In [9]:
df_draws = pd.DataFrame(columns=['Datum', '1 aus 50', '2 aus 50', '3 aus 50', '4 aus 50', '5 aus 50', '1 aus 12', '2 aus 12'])

In [10]:
for date in date_list:
    try:
        classes, five, two = get_quotes(date)
    except:
        continue

    for i in classes:
        index = len(df)
        df.loc[index, 'Datum'] = date
        df.loc[index, 'Klasse'] = i['klasse']
        df.loc[index, 'Gewinn'] = i['quote']

    index = len(df_draws)
    df_draws.loc[index, 'Datum'] = date
    df_draws.loc[index, '1 aus 50'] = five[0]
    df_draws.loc[index, '2 aus 50'] = five[1]
    df_draws.loc[index, '3 aus 50'] = five[2]
    df_draws.loc[index, '4 aus 50'] = five[3]
    df_draws.loc[index, '5 aus 50'] = five[4]
    df_draws.loc[index, '1 aus 12'] = two[0]
    df_draws.loc[index, '2 aus 12'] = two[1]   

    time.sleep(5)

In [11]:
df.to_csv('Gewinnquoten.csv', index=None)
df_draws.to_csv('ab 2022.csv', index=None)